In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import os

USE_DRIVE = False
if USE_DRIVE:
    from google.colab import drive

    drive.mount('/content/gdrive')
    ROOT_DRIVE = '/content/gdrive/MyDrive/'
    DRIVE_FOLDER_PATH = os.path.join(ROOT_DRIVE, 'HuBMAP')
    IMAGES_PATH = os.path.join(DRIVE_FOLDER_PATH, 'hubmap-256x256', 'train')
    MASKS_PATH = os.path.join(DRIVE_FOLDER_PATH, 'hubmap-256x256', 'masks')
else:
    IMAGES_PATH = os.path.join('data', '256x256', 'train')
    MASKS_PATH = os.path.join('data', '256x256', 'masks')

In [ ]:
!nvidia-smi

# DeepLab HuBMAP

In [ ]:
from utils import get_device_colab, set_deterministic_colab

NOT_COLAB = True
if not NOT_COLAB:

    DEVICE = get_device_colab()
else:
    DEVICE = 'cpu'

SEED = 42
set_deterministic_colab(SEED)

In [ ]:
from model.pretrained_models import get_deeplabv3_resnet

model = get_deeplabv3_resnet(DEVICE, resnet_layers=101).to(DEVICE)

In [ ]:
from preprocessing.dataset import get_training_validation_sets, HuBMAPDataset
from training.loop import Trainer
from training.loss_functions import BinaryDiceLoss

import torch
from torchvision import transforms

training_dataset, training_images, validation_dataset, validation_images = get_training_validation_sets(IMAGES_PATH,
                                                                                                        MASKS_PATH,
                                                                                                        0.3, 
                                                                                                        {'train': transforms.Compose([transforms.ToTensor()]), 'val': None})

criterion = BinaryDiceLoss().to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

trainer = Trainer(model,
                  0.3,
                  criterion,
                  optimizer,
                  training_dataset,
                  16,
                  "logs",
                  DEVICE)

stats = trainer.train(1, verbose=True)

In [ ]:
print(stats)